# ANU ASTR4004 2025 - Week 4 (12+14 August 2025): Preprocessing Data

Author: Dr Sven Buder (sven.buder@anu.edu.au)

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Time-yourself" data-toc-modified-id="Time-yourself-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Time yourself</a></span></li><li><span><a href="#Preprocessing-of-Data-Files" data-toc-modified-id="Preprocessing-of-Data-Files-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Preprocessing of Data Files</a></span><ul class="toc-item"><li><span><a href="#Creating-data-directories-and-downloading-files" data-toc-modified-id="Creating-data-directories-and-downloading-files-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Creating data directories and downloading files</a></span></li><li><span><a href="#Reading-CSV-(Comma-Separated-Values)-Files" data-toc-modified-id="Reading-CSV-(Comma-Separated-Values)-Files-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Reading CSV (Comma Separated Values) Files</a></span></li><li><span><a href="#FITS-Images-(the-I-in-Flexible-Image-Transport-System)" data-toc-modified-id="FITS-Images-(the-I-in-Flexible-Image-Transport-System)-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>FITS Images (the I in Flexible Image Transport System)</a></span><ul class="toc-item"><li><span><a href="#Metadata:-store-useful-information-about-your-tables" data-toc-modified-id="Metadata:-store-useful-information-about-your-tables-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>Metadata: store useful information about your tables</a></span></li><li><span><a href="#The-world-coordinate-system-at-your-fingertips" data-toc-modified-id="The-world-coordinate-system-at-your-fingertips-2.3.2"><span class="toc-item-num">2.3.2&nbsp;&nbsp;</span>The world coordinate system at your fingertips</a></span></li></ul></li></ul></li><li><span><a href="#Working-with-Data-Catalogues" data-toc-modified-id="Working-with-Data-Catalogues-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Working with Data Catalogues</a></span><ul class="toc-item"><li><span><a href="#Downloading-Data-via-ADQL/SQL" data-toc-modified-id="Downloading-Data-via-ADQL/SQL-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Downloading Data via ADQL/SQL</a></span></li><li><span><a href="#Joining-2-catalogues" data-toc-modified-id="Joining-2-catalogues-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Joining 2 catalogues</a></span></li><li><span><a href="#Cleaning-Catalogues" data-toc-modified-id="Cleaning-Catalogues-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Cleaning Catalogues</a></span></li><li><span><a href="#Scaling-entries-(e.g.-for-fitting)" data-toc-modified-id="Scaling-entries-(e.g.-for-fitting)-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Scaling entries (e.g. for fitting)</a></span></li><li><span><a href="#Visualising-targets-with-ipyaladin" data-toc-modified-id="Visualising-targets-with-ipyaladin-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Visualising targets with ipyaladin</a></span></li></ul></li><li><span><a href="#Interpolation" data-toc-modified-id="Interpolation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Interpolation</a></span></li></ul></div>

If you want to get an automatic Table of Contents as above and codefolding, you can install notebook extensions for jupyter:

```bash
pip install jupyter_contrib_nbextensions
jupyter contrib nbextension install --user
```

Then navigate to the "Nbextensions" bar when you start Jupyter and look for "Table of Contents (2)" as well as "Codefolding"  
Note: You might have to tweak a few setting to avoid numbering Notebook headers like the one above

In [ ]:
try:
    %matplotlib inline
    %config InlineBackend.figure_format='retina'
except:
    pass

import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# Make the size and fonts larger for this presentation
plt.rcParams['figure.figsize'] = (10, 8)
plt.rcParams['font.size'] = 16
plt.rcParams['lines.linewidth'] = 2

## Time yourself

In [ ]:
from functools import wraps
import time

def timeit(func):
    """Prints the runtime of decorated functions"""
    @wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()
        value = func(*args, **kwargs)
        end_time = time.perf_counter()
        run_time = end_time - start_time
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer

@timeit
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])

waste_some_time(999)

## Preprocessing of Data Files

Different researchers (and industries) prefer different data formats and even in astronomy you will encounter most of the following files:

| Name   | File Ending | Advantages                                                                 | Encounter | How to Read with Python                       |
|--------|-------------|---------------------------------------------------------------------------|-|----------------------------------------------------|
| Excel  | .xlsx       | User-friendly,<br>supports formatting,<br>widely used in business.      | Collaboration spreadsheets | `pandas.read_excel('file.xlsx')` |
| CSV    | .csv        | Lightweight,<br>human-readable,<br>easily imported/exported,<br>universal format.    | CDS VizieR | `pandas.read_csv('file.csv')`                           |
| FITS   | .fits       | Optimized for astronomy data,<br>supports metadata,<br>handles large datasets.    | Astronomy Survey Images and Catalogues | `table = astropy.table.Table.read('file.fits)`<br>`table.to_pandas()`     |
| HDF5   | .hdf5  | Efficient storage for large, complex datasets,<br>supports metadata,<br>fast I/O. | Astronomy Simulations | `pandas.read_hdf('file.h5', key='data')`                |
| Xarray | .nc         | Handles labeled n-dimensional arrays,<br>integrates with Dask for big data,<br>Well tested on TB-scale in climate modelling.    | Future Simulations/Observations? | `xarray.load_dataarray('file.nc')` |


The above table gives you a very incomplete idea of the vast amount of table formats out there. As our measurement and simulation data increases, we will move downwards. The last two file types allow you to only read in columns/rows of files or only their metadata - great if you only care about a small part in a terra-byte large file!

Currently, most astronomers are *stuck* in the lower middle of the chart - including myself. Because I work a lot with `FITS` files, I usually work with`astropy.table`.  

In terms of python packages, `pandas` is widely used in astronomy and data science. 
Switching between them is not super-difficult, so I will try to show you examples of both.

In [ ]:
import pandas as pd
from astropy.table import Table # this works well for FITS data catalogues
from astropy.io import fits # this is your more agnostic way to work for FITS images

### Creating data directories and downloading files 

You might need to download a file (if you have not already).

The `os` package comes in handy, because it allows you to check if a file or directory exists, and even create a directory if necessary!

In [ ]:
import os

# Check if 'data' directory exists, if not, create it
if not os.path.exists('data'):
    os.makedirs('data')

# You can also check if a specific file exists
file_path = 'data/reduced_TAN_C14.fits'

if not os.path.exists(file_path):
    try:
        # Download the file from a URL, if it does not exist
        import wget
        wget.download(
            url='http://data.astropy.org/tutorials/FITS-cubes/reduced_TAN_C14.fits',
            out=file_path
        )
    except Exception as e:
        raise RuntimeError(f"Could not download FITS file: {e}")

### Reading CSV (Comma Separated Values) Files

In [ ]:
# Option 1: Pandas
galah_data = pd.read_csv('data/galah_dr3_allstar_m67_lite.csv')
galah_data[:5] # let's just display the first 5 rows

In [ ]:
# Option 2: Astropy
galah_data = Table.read('data/galah_dr3_allstar_m67_lite.csv')
galah_data[:5] # let's just display the first 5 rows

### FITS Images (the I in Flexible Image Transport System)

also check out the astropy tutorials: https://learn.astropy.org/tutorials/FITS-cubes.html

#### Metadata: store useful information about your tables

Here, we will work with a FITS file from the Radio astronomy survey HI4PI.  
The specific region of the sky that was observed in this tile (C14) is the Small Magellanic Cloud (SMC).

In [ ]:
# FITS Images
fits_file = fits.open('data/reduced_TAN_C14.fits')
print(fits_file)

In [ ]:
#fits_file is now a list of objects, each one corresponding to a different extension
print('This file has {0} extensions.'.format(len(fits_file)))

In [ ]:
#we can also just dump information for this fits file
print(fits_file.info())

In [ ]:
#print the header for the Primary Extension
print(fits_file[0].header.items)

In [ ]:
# Try-except: your helper if you think something might fail
try:
    # try to read out data and header
    fits_data = fits_file[0].data
    fits_header = fits_file[0].header

except:
    print('fits_file was not yet open. Opening it now...')
    # this might have failed, if you have closed the fits_file and execute this cell out of order
    fits_file = fits.open('data/reduced_TAN_C14.fits')
    fits_data = fits_file[0].data
    fits_header = fits_file[0].header
    
# don't forget to close the fits_file to free up memory
fits_file.close()

#### The world coordinate system at your fingertips

In [ ]:
# The pixels are reported in Galactic longitude and latitude
# Imagine you want to translate to Right Ascension and Declination?

# GLON and GLAT come in AXIS1 and AXIS2
naxis1 = fits_header['NAXIS1']  # Number of pixels along the 1st axis
naxis2 = fits_header['NAXIS2']  # Number of pixels along the 2nd axis

# It is good practice to report FITS measurements in an agreed reference frame, 
# the World Coordinate System (WCS)
from astropy.wcs import WCS
wcs = WCS(fits_header, naxis=2)

In [ ]:
# Imagine you want to just zoom in onto the SMC: 
import astropy.units as u

# Define the desired latitude and longitude range
lat_range = [-46, -40] * u.deg  # GLAT range
lon_range = [306, 295] * u.deg  # GLON range

# Using the WCS object to convert world coordinates (GLON, GLAT) to pixel coordinates

# Convert (GLON_min, GLAT_min) and (GLON_max, GLAT_max) to pixel coordinates
glon_min_pix, glat_min_pix = wcs.world_to_pixel_values(lon_range[0], lat_range[0])  # Lower-left corner
glon_max_pix, glat_max_pix = wcs.world_to_pixel_values(lon_range[1], lat_range[1])  # Upper-right corner

# Convert to integer pixel coordinates for slicing
glon_min_pix = int(np.floor(glon_min_pix))
glon_max_pix = int(np.ceil(glon_max_pix))
glat_min_pix = int(np.floor(glat_min_pix))
glat_max_pix = int(np.ceil(glat_max_pix))

sub_wcs = wcs.deepcopy()  # Make a copy of the original WCS
sub_wcs.wcs.crpix[0] -= glon_min_pix  # Adjust reference pixel for GLON
sub_wcs.wcs.crpix[1] -= glat_min_pix  # Adjust reference pixel for GLAT

sub_data_cube = fits_data[:, glat_min_pix:glat_max_pix, glon_min_pix:glon_max_pix] * u.K

In [ ]:
# Let's estimate the 0th and 1st Moments, i.e. Intensity Map and Intensity-weighted Velocity Map 

cdelt3 = fits_header['CDELT3'] * u. m / u.s  # Velocity increment per channel (e.g., km/s)
crval3 = fits_header['CRVAL3'] * u. m / u.s  # Velocity value at the reference pixel
crpix3 = fits_header['CRPIX3']  # Reference pixel (1-based index in FITS)

# Create the velocity axis (1D array for velocity channels)
n_channels = sub_data_cube.shape[0]  # Number of velocity channels
velocity_axis = ((np.arange(n_channels) - (crpix3 - 1)) * cdelt3 + crval3)

# Select velocity between -300 and 300 km/s
velocity_select = (velocity_axis > -300 * u.km/u.s) & (velocity_axis < 302 * u.km/u.s)

# Calculate the zeroth moment: integral of I delta_v = sum(I * cdelt3)
intensity_sum = np.nansum(sub_data_cube[velocity_select,:,:], axis=0) * cdelt3

# Calculate the first moment (intensity-weighted mean velocity)
# i.e. weighted sum of velocities / sum of intensities

# Weighted sum of velocities = sum(I * velocity-center * cdelt3)
# Note: velocity center has to be offset by half a bin from velocity_axis
weighted_velocity_sum = np.nansum(sub_data_cube[velocity_select,:,:] * (velocity_axis[velocity_select, np.newaxis, np.newaxis] + 0.5 * cdelt3) * cdelt3, axis=0)

# Calculate the first moment (intensity-weighted mean velocity)
intensity_weighted_mean_velocity = np.where(intensity_sum > 0, weighted_velocity_sum / intensity_sum, np.nan)

# Convert Intensity sum to a Column Density assuming optically thin media
# Following Eq. (2) of this paper: https://www.aanda.org/articles/aa/pdf/2016/10/aa29178-16.pdf
hi_column_density = (intensity_sum * 1.823 * 10**18 / (u.cm * u.cm) * u.s / u.K / u.km).to(1/u.cm**2)

If you want to read more about this, check out the HI4PI paper: https://ui.adsabs.harvard.edu/abs/2016A%26A...594A.116H/abstract  
(or ask Naomi McClure-Griffiths here at ANU, one of the co-authors, and lead of the Parkes Radio observations of this amazing project)

In [ ]:
# Let's do some fun plotting! 

fig, ax = plt.subplots(1,1,figsize=(10, 5),subplot_kw={'projection': sub_wcs})

# image = ax.imshow(data_slice, origin='lower', cmap='RdBu_r', aspect='auto')
image = ax.imshow(intensity_weighted_mean_velocity.to(u.km/u.s).value, origin='lower', cmap='RdBu_r', vmin=0, vmax=200)

# Add a colorbar
cbar = plt.colorbar(image, pad = 0.15)
cbar.set_label(r'Line-of-Sight Velocity / $\mathrm{km\,s^{-1}}$', size=16)

# Add axes labels
ax.set_xlabel("Galactic Longitude / deg", fontsize=16)
ax.set_ylabel("Galactic Latitude / deg", fontsize=16)
ax.set_ylim(ax.get_ylim()[::-1])

# Overplot column density contours 
levels = (1e20, 5e20, 1e21, 3e21, 5e21, 7e21, 1e22)  # Define contour levels to use
ax.contour(hi_column_density.value, cmap='Greys_r', alpha=0.5, levels=levels)

# Overlay set of RA/Dec Axes
overlay = ax.get_coords_overlay('fk5')
overlay.grid(color='white', ls='dotted', lw=2)
overlay[0].set_axislabel('Right Ascension (J2000) / deg', fontsize=16)
overlay[1].set_axislabel('Declination (J2000) / deg', fontsize=16)

contour_legend = matplotlib.lines.Line2D([0], [0], color='gray', lw=2, label='HI Column Density Contours')
ax.legend(handles=[contour_legend], loc='lower left', fontsize=12)

plt.tight_layout()
plt.savefig('figures/column_density.png',dpi=200,bbox_inches='tight')
plt.show()
plt.close()

# This is adjusted from a tutorial of the astropy collaboration: https://learn.astropy.org/tutorials/FITS-cubes.html

In [ ]:
# Let's do some fun plotting! 

fig, ax = plt.subplots(1,1,figsize=(10, 6),subplot_kw={'projection': sub_wcs})

# image = ax.imshow(data_slice, origin='lower', cmap='RdBu_r', aspect='auto')
image = ax.imshow(intensity_weighted_mean_velocity.to(u.km/u.s).value, origin='lower', cmap='RdBu_r', vmin=0, vmax=200)

# Add a colorbar
cbar = plt.colorbar(image, pad = 0.05)
cbar.set_label(r'Line-of-Sight Velocity / $\mathrm{km\,s^{-1}}$', size=16)

# Add axes labels
ax.set_xlabel("Galactic Longitude / deg", fontsize=16)
ax.set_ylabel("Galactic Latitude / deg", fontsize=16)
ax.set_ylim(ax.get_ylim()[::-1])

# Overplot column density contours 
# levels = (1e20, 5e20, 1e21, 3e21, 5e21, 7e21, 1e22)  # Define contour levels to use
levels = (0.1, 0.5, 1, 3, 5, 7, 10)  # Define contour levels to use
contour = ax.contour((hi_column_density / 10**21).value, cmap='Greys_r', alpha=0.5, levels=levels)
# Add a colorbar for the column density contours
cbar2 = plt.colorbar(contour, ax=ax, orientation='horizontal')
cbar2.set_label(r'HI Column Density / $10^{21}\,\mathrm{cm^{-2}}$', size=16)

# Overlay set of RA/Dec Axes
overlay = ax.get_coords_overlay('fk5')
overlay.grid(color='white', ls='dotted', lw=2)
overlay[0].set_axislabel('Right Ascension (J2000) / deg', fontsize=16)
overlay[1].set_axislabel('Declination (J2000) / deg', fontsize=16)

# contour_legend = matplotlib.lines.Line2D([0], [0], color='gray', lw=2, label='HI Column Density Contours')
# ax.legend(handles=[contour_legend], loc='lower left', fontsize=12)

plt.tight_layout()
plt.savefig('figures/column_density2.png',dpi=200,bbox_inches='tight')
plt.show()
plt.close()

# This is adjusted from a tutorial of the astropy collaboration: https://learn.astropy.org/tutorials/FITS-cubes.html

## Working with Data Catalogues

### Downloading Data via ADQL/SQL

Remember how we read in a data table from the GALAH survey above with spectroscopic measurements?

The table has the identifier of the 1.8 billion source large Gaia DR3 catalogue in it.

Downloading the full Gaia DR3 catalogue is not useful, but we can use the `astroquery` package to perform a query with the Astronomical Data Query Language (ADQL), an extension of the Structured Query Language (SQL) to include functions when querying data.

Let's try to download the matches in Gaia DR3!

In [ ]:
from astroquery.gaia import Gaia

In [ ]:
# Extract only the Gaia DR3 source IDs for crossmatching
galah_data['source_id'] = galah_data['dr3_source_id']
gaia_source_ids = galah_data['dr3_source_id'].tolist()

# Convert the source IDs to an Astropy table to use in the query (to not upload too much data)
# with the actual identifier in Gaia DR3 (source_id)
gaia_source_ids_table = Table([gaia_source_ids], names=['source_id'])

# Define and execute the ADQL query to crossmatch with Gaia DR3
query = f"""
SELECT * 
FROM gaiadr3.gaia_source AS gaia
JOIN TAP_UPLOAD.t1 AS galah
ON gaia.source_id = galah.source_id
"""

# Upload the source_id table for crossmatching
job = Gaia.launch_job_async(query=query, upload_resource=gaia_source_ids_table, upload_table_name="t1")
gaiadr3_match = job.get_results()

In [ ]:
gaiadr3_match[:5]

### Joining 2 catalogues



In [ ]:
# fix missing keyword
galah_data['source_id'] = galah_data['dr3_source_id']

In [ ]:
from astropy.table import join

gaia_dr3_galah = join(gaiadr3_match, galah_data, keys='source_id')

In [ ]:
gaia_dr3_galah

In [ ]:
def plot_cmd_and_kiel(data, colormap_left = 'snr_c2_iraf', colormap_left_label = 'GALAH DR3 SNR CCD2'):

    # compare Gaia DR3 and GALAH DR3 measurements
    f, gs = plt.subplots(1, 2, figsize=(10,4))

    # Left panel (Color-Magnitude Diagram, CMD)
    ax = gs[0]
    ax.text(0.05, 0.95, 'a)', transform=ax.transAxes, fontsize=14, ha = 'left', va='top')

    sc = ax.scatter(
        data['bp_rp'],
        data['phot_g_mean_mag'] + 5 * np.log10(data['parallax']/100.), 
        c=data[colormap_left],
        cmap='viridis', s=10
    )
    ax.invert_yaxis()
    ax.set_xlabel(r'$G_\mathrm{BP} - G_\mathrm{RP}~/~\mathrm{mag}$')
    ax.set_ylabel(r'$G~/~\mathrm{mag}$')
    ax.set_title(r'$Gaia$ DR3')

    # Adding a colorbar for SNR in the left panel
    cbar = plt.colorbar(sc, ax=ax)
    cbar.set_label(colormap_left_label)

    # Right panel (Teff-logg diagram)
    ax = gs[1]
    ax.text(0.05, 0.95, 'b)', transform=ax.transAxes, fontsize=14, ha = 'left', va='top')
    sc2 = ax.scatter(
        data['teff'],
        data['logg'], 
        c=data['fe_h'],
        cmap='plasma', s=10
    )
    ax.invert_xaxis()
    ax.invert_yaxis()
    ax.set_xlabel(r'$T_\mathrm{eff}~/~\mathrm{K}$')
    ax.set_ylabel(r'$\log (g~/~\mathrm{cm\,s^{-2}}$')
    ax.set_title('GALAH DR3')

    # Adding a colorbar for [Fe/H] in the right panel
    cbar2 = plt.colorbar(sc2, ax=ax)
    cbar2.set_label('[Fe/H]')

    plt.tight_layout()
    plt.show()
    plt.close()
    
plot_cmd_and_kiel(data=gaia_dr3_galah)

### Cleaning Catalogues

Quite often, catalogues do not have all measurements.

Just above, you have seen the error message
```
RuntimeWarning: invalid value encountered in log10
gaia_dr3_galah['phot_g_mean_mag'] + 5 * np.log10(gaia_dr3_galah['parallax']/100.),
```

This message is expected if np.log10() is applied to a value that is not positive (this could be either negative value or value that is "Not a Value" aka NaN). Let's get a first idea:

In [ ]:
gaia_dr3_galah['parallax']/100.

We already see negative values, but we can also imagine values that are *Not a Number* aka NaN or they might have a bitmask "flag" that indicates their quality.

Negative parallax measurements are true measurements! They just tell us that the source is quite far away. Later in this course how we can use our **prior** knowledge that distances from us have to be positive to still extract something useful out of these measures.

For now, we will simply identify these measurements and not use them to avoid error messages.
For a research paper, this would be a selection cut that has needs to be documented for reproducability!

**NaN entries**  

You can identify NaN entries with the check, e.g. effective temperature $T_\mathrm{eff}$.
If you have checked if a value is NaN, you can also invert the result with a `~` (switch True <-> False):

In [ ]:
check_if_parallax_values_finite = np.isfinite(gaia_dr3_galah['parallax'])
check_if_parallax_values_nan = np.isnan(gaia_dr3_galah['parallax'])
check_if_parallax_values_not_fintie = ~np.isfinite(gaia_dr3_galah['parallax'])
check_if_parallax_positive = gaia_dr3_galah['parallax'] > 0

# you can also use the np.where function:
where_parallax_not_positive = np.where(~check_if_parallax_positive==True)

In [ ]:
gaia_dr3_galah['parallax'][where_parallax_not_positive]

**bitmask flags**

A bitmask is a way of storing multiple boolean (True/False) values in a single integer by representing each condition with a different bit in the binary representation of the number. For example, you might use bitmasks in a catalog to encode several flags in a compact form.

You can for example imagine a bitmask flag `0111`, which would add up to `8*0 + 4*1 + 2*1 + 1*1 = 7`. So with just 1 number, you can check for 4 different things!

You can also check if just a specific bitmask is raised:

In [ ]:
def is_bit_raised(flag, bit):
    """
    Check if *flag* has the value for a specific *bit* raised
    """
    return (flag & bit) != 0

In GALAH DR3, one of these quality flags is `flag_sp`, which includes a lot of details about quality checks and extra information about a star, for example, if we think it is a binary star.

According to the documentation (Table 4 from https://ui.adsabs.harvard.edu/abs/2021MNRAS.506..150B)
that would mean raised flags 32 (spectroscopic binary) or 64 (photometric binary).

We can find the stars that are flagged as binaries with an `|` check, which means `or`.
You could also find stars that are spectroscopic *and* photometric binaries by replacing `|` with `&`:

In [ ]:
check_if_binary = (
    is_bit_raised(gaia_dr3_galah['flag_sp'], 32) &
    is_bit_raised(gaia_dr3_galah['flag_sp'], 64)
)

gaia_dr3_galah['binary'] = check_if_binary

In [ ]:
plot_cmd_and_kiel(
    gaia_dr3_galah[check_if_parallax_positive],
    colormap_left='binary',
    colormap_left_label='Binary True/False'
)

### Scaling entries (e.g. for fitting)

You can imagine that any type of optimisation of a function has to start with an initial value.
In some cases, this value might be off by quite a lot and the fitting algorithm might be stuck, or take a lot of time to get close to the final result.

There are 2 ways to improve your situation:
1. Start with a better initial guess  
2. Rescale your values, so that they are between -1 and 1 or scaled to their standard deviation  

Take a look at: https://scikit-learn.org/stable/modules/preprocessing.html

### Visualising targets with ipyaladin

You can find ipyaladin at https://github.com/cds-astro/ipyaladin

In [ ]:
from ipyaladin import Aladin
aladin = Aladin()
aladin

# Initialize Aladin and set the view to M67 in 2MASS J color image
aladin = Aladin(target='M67', survey='2MASS-J', fov=0.5)  # Adjust the field of view as needed

# Display Aladin widget
aladin

## Interpolation

In [ ]:
from scipy.interpolate import CubicSpline
from scipy.interpolate import interp1d

# sample sparse data points from a sine function
sample_x = np.linspace(-np.pi, np.pi, 7)
sample_y = np.sin(sample_x)
plt.plot(sample_x, sample_y, 'ko')

# the true sine function
z = np.linspace(np.min(sample_x), np.max(sample_x), 100)
plt.plot(z, np.sin(z), label='real')

# linear interpolation
f1 = interp1d(sample_x, sample_y)
plt.plot(z, f1(z), label='interp1d')

# cubic spline interpolation
f2 = CubicSpline(sample_x, sample_y)
plt.plot(z, f2(z), label='CubicSpline')

# add legend
plt.legend();
plt.grid(linestyle='--');

In [ ]:
# let's plot the residual
plt.plot(z, f1(z)-np.sin(z), label='interp1d residuals', color='orange')
plt.plot(z, f2(z)-np.sin(z), label='CubicSpline residuals', color='green')
plt.legend();
plt.grid(linestyle='--');

In [ ]:
import numpy as np
from scipy.interpolate import RegularGridInterpolator
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Example: 3D linear interpolation

# Define the grid points in each dimension
x = np.linspace(0, 10, 5)  # 5 points in the x direction
y = np.linspace(0, 10, 5)  # 5 points in the y direction
z = np.linspace(0, 10, 5)  # 5 points in the z direction

# Create a 3D grid of values (you can replace this with actual data)
# For demonstration, we'll just use the sum of coordinates for simplicity
values = np.zeros((len(x), len(y), len(z)))

for i in range(len(x)):
    for j in range(len(y)):
        for k in range(len(z)):
            values[i, j, k] = x[i] + y[j] + z[k]

# Create the interpolator object
interpolator = RegularGridInterpolator((x, y, z), values)

# Generate a finer 3D grid for visualization using interpolation
x_fine = np.linspace(0, 10, 30)
y_fine = np.linspace(0, 10, 30)
z_fine = np.linspace(0, 10, 30)

x_fine_mesh, y_fine_mesh, z_fine_mesh = np.meshgrid(x_fine, y_fine, z_fine)

# Prepare points for interpolation
fine_points = np.array([x_fine_mesh.ravel(), y_fine_mesh.ravel(), z_fine_mesh.ravel()]).T
fine_values = interpolator(fine_points).reshape(x_fine_mesh.shape)

# Plotting
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot the original grid points in red
x_grid, y_grid, z_grid = np.meshgrid(x, y, z, indexing='ij')
ax.scatter(x_grid, y_grid, z_grid, color='red', s=50, label='Original Grid Points')

# Plot interpolated values as a scatter plot on the fine grid
scatter = ax.scatter(x_fine_mesh, y_fine_mesh, z_fine_mesh, c=fine_values, cmap='viridis', alpha=0.7)

# Add color bar for interpolated values
colorbar = fig.colorbar(scatter, ax=ax, pad=0.1, label='Interpolated Value')

# Labels and title
ax.set_title('3D Linear Interpolation')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.legend()
plt.show()


In [ ]:
import numpy as np
from scipy.interpolate import LinearNDInterpolator
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Example: 3D interpolation on a non-regular grid

# Generate random non-regular grid points in 3D space
np.random.seed(42)
num_points = 30  # Number of scattered data points
x = np.random.uniform(0, 10, num_points)
y = np.random.uniform(0, 10, num_points)
z = np.random.uniform(0, 10, num_points)

# Values at the random grid points (for example, we'll use the sum of the coordinates)
values = x + y + z

# Create the LinearNDInterpolator object for scattered data
interpolator = LinearNDInterpolator(list(zip(x, y, z)), values)

# Generate a finer grid for visualization (in 3D space)
x_fine = np.linspace(0, 10, 30)
y_fine = np.linspace(0, 10, 30)
z_fine = np.linspace(0, 10, 30)

# Meshgrid for finer points
x_fine_mesh, y_fine_mesh, z_fine_mesh = np.meshgrid(x_fine, y_fine, z_fine)

# Prepare the fine points for interpolation
fine_points = np.array([x_fine_mesh.ravel(), y_fine_mesh.ravel(), z_fine_mesh.ravel()]).T

# Perform interpolation at the fine points
fine_values = interpolator(fine_points)

# Reshape the results to match the fine grid shape
fine_values = fine_values.reshape(x_fine_mesh.shape)

# Plotting
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot the original scattered grid points in red
ax.scatter(x, y, z, color='red', s=50, label='Original Scattered Points')

# Plot the interpolated values as a scatter plot on the fine grid
# Mask invalid (NaN) values that could occur where interpolation is not defined
valid_mask = ~np.isnan(fine_values)
scatter = ax.scatter(
    x_fine_mesh[valid_mask],
    y_fine_mesh[valid_mask],
    z_fine_mesh[valid_mask],
    c=fine_values[valid_mask],
    cmap='viridis',
    alpha=0.7,
)

# Add a color bar to represent the interpolated values
colorbar = fig.colorbar(scatter, ax=ax, pad=0.1, label='Interpolated Value')

# Labels and title
ax.set_title('3D Interpolation on a Non-Regular Grid')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.legend()
plt.show()
